In [1]:
import os
import gzip
import numpy as np
import json
import networkx as nx

from utilities import qubo_to_ising, matrix_to_couplings

In [12]:
Ntot = 200
Ngraphs = 50
np.random.seed(42)

for idx in range(Ngraphs):
    # Create dense graph
    G = nx.complete_graph(Ntot)

    edges = list(G.edges)
    weights = {e: int(np.random.choice([-1, 1])) for e in edges}

    # Create J_ising dict 
    J_ising = {}
    for (i, j), w in weights.items():
        J_ising[(i, j)] = w

    Q = np.zeros((Ntot, Ntot))
    for i in range(Ntot):
        for j in range(Ntot):
            # Get J[i, j] or J[j, i] depending on order
            if (i, j) in J_ising:
                J_ij = J_ising[(i, j)]
            elif (j, i) in J_ising:
                J_ij = J_ising[(j, i)]
            else:
                J_ij = 0
            Q[i, j] = 2 * J_ij
            Q[j, i] = 2 * J_ij 
        Q[i, i] = -2 * sum(J_ising.get((i, k), J_ising.get((k, i), 0)) for k in range(Ntot) if k != i)

    # Add constant 
    offset = sum(weights.values())

    # Convert to QUBO and Ising couplings
    J, h = matrix_to_couplings(Q)

    # Save graph
    os.makedirs(f"../data/sk_model/{Ntot}v/graphs", exist_ok=True)
    with open(f"../data/sk_model/{Ntot}v/graphs/graph{idx}.json", "w") as f:
        # edge and weight
        json.dump({
            "N": Ntot, 
            "edges": {f"{k[0]},{k[1]}": v for k, v in weights.items()}
            }, f, indent=4)

    # Save QUBO
    os.makedirs(f"../data/sk_model/{Ntot}v/QUBO", exist_ok=True)
    with open(f"../data/sk_model/{Ntot}v/QUBO/qubo_graph{idx}.json", "w") as f:
        json.dump({
            "J": {f"{k[0]},{k[1]}": v for k, v in J.items()}, 
            "h": {str(k): v for k, v in h.items()}, 
            "c": offset
            }, f, indent=4)

    # Save Ising
    os.makedirs(f"../data/sk_model/{Ntot}v/Ising", exist_ok=True)
    with open(f"../data/sk_model/{Ntot}v/Ising/ising_graph{idx}.json", "w") as f:
        json.dump({
            "J": {f"{k[0]},{k[1]}": v for k, v in J_ising.items()}
        }, f, indent=4)

In [36]:
x = [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1]
z = [1 if xi == 0 else -1 for xi in x]
value = 0
for (i, j), Jij in J_ising.items():
    value += Jij * z[i] * z[j]
print(value)

value_qubo = 0
for (i, j), Jij in J.items():
    value_qubo += Jij * x[i] * x[j]
for i, hi in h.items():
    value_qubo += hi * x[i]
value_qubo += offset
print(value_qubo)

-221
-221.0
